# 2.2.6 Filtrado de spam en mensajes de texto SMS

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.naive_bayes import BernoulliNB

from sklearn.metrics import confusion_matrix

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import sys
sys.path.append("../../../../../") 

from utils.paths import make_dir_line

modality = 'u'
project = 'Analitica predictiva'
data = make_dir_line(modality, project)

raw = data('raw')

Los clasificadores Bayesianos ingenuos son herramientas de gran utilidad para la construcción de sistemas de clasificación, como ya se discutio en los tutoriales anteriores. En este tutorial se utiliza un clasificador Bayesiano ingenuo para determinar si un mensaje SMS es válido o es spam.

## Definición del problema

a recepción de publicidad no deseada a traves mensajes de texto usando SMS (Short Message Service) es un problema que afecta a muchos usuarios de teléfonos móviles. El problema radica en que los usuarios deben pagar por los mesajes recibidos, y por este motivo resulta muy importante que las compañías prestadoras del servicio puedan filtrar mensajes indeseados antes de enviarlos a su destinatario final. Los mensajes tienen una longitud máxima de 160 caracteres, por lo que el texto resulta poco para realizar la clasificación, en comparación con textos más largos (como los emails). Adicionalmente, los errores de digitación dificultan el proceso de detección automática.

La muestra contiene 5574 mensajes en inglés, no codificados y clasificados como legítimos (ham) o spam (http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). El problema en términos de los datos consiste en clasificar si un mensaje SMS es legítico o spam, a partir del análisis de las palabras que contiente, partiendo del supuesto de que ciertas palabras que son más frecuentes dependiendo del tipo de mensaje. Esto implica que en la fase de preparación de los datos se deben extraer las palabras que contiene cada mensaje para poder realizar el análsis.

## Carga de datos

In [ ]:
df = pd.read_csv(
    raw / "sms-spam.csv",
    sep=",",
    encoding="latin-1",
)

df.head()

In [ ]:
#
# Verifica la lectura de los datos
#
df.describe()

## Conteo de cantidad de mensajes por tipo

In [ ]:
#
# Se obtiene la cantidad de casos para
# cada tipo de mensaje.
#
df.type.value_counts()

In [ ]:
df.type.value_counts().plot.bar();

In [ ]:
#
# Se convierte el conteo anterior en probabilidades.
#
round(100 * df.type.value_counts() / sum(df.type.value_counts()), 1)

## Stemmer

In [ ]:
#
# Se construye un stemmer que reduce una palabra a su raiz o 'stem'.
# {llorar, lloramos, lloraron} -> llorar
# {biblioteca, bibliotecario} -> bibliotec
#
stemmer = PorterStemmer()
df["stemmed"] = df.text.apply(lambda x: " ".join([stemmer.stem(w) for w in x.split()]))
df.head(10)

### Matriz de Términos del Documento

In [ ]:
#
# Stopwords:
#   i  me  my  myself  we  our  ours  ourselves ....
#   am  is  are  was  were  be  been  being  have
#   has  had  having  do  does  did  doing  ...
#   a  an  the  and  but  if  or  because  as  until
#   while  of  at  by  for  with  about  against ...
#   ...
#
# token_pattern:
# https://docs.python.org/3/howto/regex.html#regex-howto
#
#   \w cualquier caracter alfanumerico [a-zA-Z0-9_]
#   \w\w+ cadenas de dos o mas caracteres
#   \b  word boundary
#
count_vect = CountVectorizer(
    analyzer="word",                # a nivel de palabra
    lowercase=True,                 # convierte a minúsculas
    stop_words="english",           # stop_words en inglés
    token_pattern=r"(?u)\b\w\w+\b", # patrones a reconocer
    binary=True,                    # Los valores distintos de cero son fijados en 1
    max_df=1.0,                     # máxima frecuencia a considerar
    min_df=5,                       # ignora palabras con baja frecuencia
)

#
# Aplica la función al texto
#
dtm = count_vect.fit_transform(df.stemmed)

#
# Las filas contienen los mensajes
# y las clomunas los términos
#
dtm.shape

In [ ]:
#
# Palabras aprendidas de los mensajes de texto
#
vocabulary = count_vect.get_feature_names_out()
len(vocabulary)

In [ ]:
#
# Primeras palabras del vocabulario
#
vocabulary[0:10]

In [ ]:
#
# Se puede mejorar diciendo que solo se reconozcan
# palabras formadas por letras
#
count_vect = CountVectorizer(
    analyzer="word",
    lowercase=True,
    stop_words="english",
    token_pattern=r"(?u)\b[a-zA-Z][a-zA-Z]+\b",
    binary=True,
    max_df=1.0,
    min_df=5,
)

dtm = count_vect.fit_transform(df.stemmed)

#
# Las filas contienen los mensajes
# y las clomunas los términos
#
dtm.shape

In [ ]:
vocabulary = count_vect.get_feature_names_out()
vocabulary[0:10]

In [ ]:
#
# Recupera los mensajes de la dtm
#
def dtm2words(dtm, vocabulary, index):
    as_list = dtm[index, :].toarray().tolist()
    docs = []
    for i in index:
        k = [vocabulary[iword] for iword, ifreq in enumerate(as_list[i]) if ifreq > 0]
        docs += [k]
    return docs


for i, x in enumerate(dtm2words(dtm, vocabulary, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])):
    print("Org: ", df.text[i])
    print("Mod: ", " ".join(x))
    print("")

### Conjuntos de entrenamiento y prueba

In [ ]:
#
# Creación de los conjuntos de entrenamiento y prueba.
#
X_train = dtm[
    0:4168,
]
X_test = dtm[
    4169:,
]

y_train_true = df.type[0:4168]
y_test_true = df.type[4169:]

#
# Distribución de los datos en el conjunto de entrenamiento.
#
round(100 * y_train_true.value_counts() / sum(y_train_true.value_counts()), 1)

In [ ]:
#
# Distribución de los datos en el conjunto de entrenamiento.
#
round(100 * y_test_true.value_counts() / sum(y_test_true.value_counts()), 1)

### Entrenamiento del modelo

In [ ]:
#
# Se crea un clasificador Naive Bayes (NB)
#
clf = BernoulliNB()

#
# Se entrena el clasificador
#
clf.fit(X_train.toarray(), y_train_true)
clf

### Evaluación del modelo

In [ ]:
#
# Se pronostica para los datos de prueba.
#
y_test_pred = clf.predict(X_test.toarray())
y_test_pred_prob = clf.predict_proba(X_test.toarray())
y_test_pred

In [ ]:
#
# Métricas de desempeño
#

confusion_matrix(y_true=y_test_true, y_pred=y_test_pred)

#
# Pronostico en las columnas
# Real en las filas
#

In [ ]:
clf.predict_proba(X_test.toarray())

In [ ]:
#
# Resulta más conveniente preparar una nueva tabla que
# muestre la clasificación y no únicamente las
# probabilidades.
#
results = pd.DataFrame(
    data={
        "actual_type": y_test_true,
        "predict_type": y_test_pred,
        "prob_ham": [v[0] for v in y_test_pred_prob],
        "prob_spam": [v[1] for v in y_test_pred_prob],
    }
)

results.head(5)

In [ ]:
#
# Mensajes con clasificación errónea.
# Resulta muy importante determinar porque los
# mensajes están mal clasificados
#
results[results["actual_type"] != results["predict_type"]]

In [ ]:
#
# Sin embargo, es mucho más intersante extraer
# mensajes con probabilidades numéricamente
# cercanas a 0.5. Estos podrían generar ambiguedad
# en la clasificación.
#
results[(results["prob_spam"] > 0.4) & (results["prob_spam"] < 0.6)]

In [ ]:
#
# Mensajes mal clasificados con probabilidad cercana a 0.5
#
results[
      (results["prob_spam"] > 0.4)
    & (results["prob_spam"] < 0.6)
    & (results["actual_type"] != results["predict_type"])
]

In [ ]:
print('ok_')